In [5]:
import torch
import numpy as np
from torchvision import transforms
from custom_transforms import  NormalizeSpectrogram, ToTensor, InterpolateSpectrogram
from SearchNet import SearchNet

def predict_single_spectrogram(spectrogram_array, device):
    # Initialize model
    model = SearchNet(0.10, 1.16, 0.75, 0.75).to(device)
    
    # Load trained weights
    model.load_state_dict(torch.load("final_model_final_model"))
    model.eval()  # Set to evaluation mode
    
    # Set up transforms (same as in training)
    transform = transforms.Compose([
        NormalizeSpectrogram(),
        ToTensor(),
        InterpolateSpectrogram()
    ])
    
    # Transform your input
    # Assuming spectrogram_array is your (79, 534, 6) numpy array
    transformed_input = transform({"spectrogram": spectrogram_array})
    
    # Add batch dimension
    input_tensor = transformed_input["spectrogram"].unsqueeze(0).to(device)
    
    # Make prediction
    with torch.no_grad():
        prediction = model(input_tensor)
    
    return prediction.squeeze().cpu().numpy()

In [6]:
DEVICE = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"

spectrogram_array = np.load(r"ball_hit_spectrograms.npy")

prediction = predict_single_spectrogram(spectrogram_array, DEVICE)

print(f'Predicted Radial Velocity: {prediction:.3f}')

C:\Users\gtoft\AppData\Local\Temp\ipykernel_15636\869582734.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("final_model_final_model"))

TypeError: unhashable type: 'slice'